In [3]:
!pip install selenium

In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time


In [ ]:
ques-1

In [13]:
def search_amazon(product):
    driver = webdriver.Chrome()
    driver.get(f"https://www.amazon.in/s?k={product}")
    results = driver.find_elements(By.XPATH,"//span[@class='a-size-medium a-color-base a-text-normal']")

search_amazon("guitar")

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

def scrape_amazon_products(search_query):
    # Set up the webdriver
    driver = webdriver.Chrome()
    driver.get('https://www.amazon.in/')

    # Enter the search query and submit the form
    search_box = driver.find_element(By.ID, 'twotabsearchtextbox')
    search_box.send_keys(search_query)
    search_box.submit()

    # Wait for the page to load
    time.sleep(5)

    # Extract product data
    products = []
    for page_num in range(1, 4):
        # Extract data for each product on the current page
        product_elements = driver.find_elements(By.XPATH, '//div[@data-index]/div/span/div/div')
        for product_element in product_elements:
            # Extract brand name
            try:
                brand_name_element = product_element.find_element(By.XPATH, './/div[2]/div[1]/div[1]/div[1]/h5/a')
                brand_name = brand_name_element.text.strip()
            except:
                brand_name = '-'

            # Extract name of the product
            try:
                name_element = product_element.find_element(By.XPATH, './/div[2]/div[1]/div[1]/div[1]/h2/a/span')
                name = name_element.text.strip()
            except:
                name = '-'

            # Extract price
            try:
                price_element = product_element.find_element(By.XPATH, './/div[2]/div[2]/div[1]/div[1]/a/span/span')
                price = price_element.text.strip()
            except:
                price = '-'

            # Extract return/exchange policy
            try:
                return_exchange_element = product_element.find_element(By.XPATH, './/div[2]/div[3]/div/div/span')
                return_exchange = return_exchange_element.text.strip()
            except:
                return_exchange = '-'

            # Extract expected delivery date
            try:
                expected_delivery_element = product_element.find_element(By.XPATH, './/div[2]/div[4]/div/div/span')
                expected_delivery = expected_delivery_element.text.strip()
            except:
                expected_delivery = '-'

            # Extract availability
            try:
                availability_element = product_element.find_element(By.XPATH, './/div[2]/div[5]/span')
                availability = availability_element.text.strip()
            except:
                availability = '-'

            # Extract product URL
            try:
                url_element = product_element.find_element(By.XPATH, './/div[2]/div[1]/div[1]/div[1]/h2/a')
                url = url_element.get_attribute('href').strip()
            except:
                url = '-'

            # Store the data in a dictionary and add it to the list of products
            product_data = {
                'brand_name': brand_name,
                'name': name,
                'price': price,
                'return_exchange': return_exchange,
                'expected_delivery': expected_delivery,
                'availability': availability,
                'url': url,
            }
            products.append(product_data)

        # Go to the next page if it exists
        try:
            next_page_button = driver.find_element(By.XPATH, '//span[@class="s-pagination-item s-pagination-next"]')
            next_page_button.click()
        except:
            break

    # Close the webdriver
    driver.close()

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(products)

    # Save the DataFrame to a CSV file
    df.to_csv(f'{search_query}_products.csv', index=False)

    # Return the results
    return df

# Example usage
search_query = input('Enter a search query: ')
df = scrape_amazon_products(search_query)
print(df)


Enter a search query: guitar
  brand_name name price return_exchange expected_delivery availability url
0          -    -     -               -                 -            -   -
1          -    -     -               -                 -            -   -


ques-3

In [5]:
from selenium import webdriver
import urllib.request

def scrape_images():
    driver = webdriver.Chrome()
    keywords = ['fruits', 'cars', 'Guitar', 'Cakes']
    for keyword in keywords:
        url = f"https://www.google.com/search?q={keyword}&tbm=isch"
        driver.get(url)
        images = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
        for i, image in enumerate(images[:10]):
            image_url = image.get_attribute('src')
            urllib.request.urlretrieve(image_url, f"{keyword}{i}.jpg")
    driver.quit()
    print("Images scraped successfully!")

scrape_images()

Images scraped successfully!


In [ ]:
ques-4

In [2]:


def search_flipkart(smartphone):
    # Set up the web driver
    driver = webdriver.Chrome()
    driver.get("https://www.flipkart.com")
    
    # Close the login popup if it appears
    close_button = driver.find_element(By.XPATH, "//button[@class='_2KpZ6l _2doB4z']")
    if close_button.is_displayed():
        close_button.click()
    
    # Search for the smartphone
    search_box = driver.find_element(By.XPATH, "//input[@title='Search for products, brands and more']")
    search_box.send_keys(smartphone)
    search_box.send_keys(Keys.RETURN)
    
    # Wait for the page to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='_1YokD2 _3Mn1Gg']")))
    
    # Scrape the details of the products listed on the first page of search results
    data = []
    products = driver.find_elements(By.XPATH, "//div[@data-id]")
    
    for product in products:
        details = {}
        
        title = product.find_element(By.XPATH, ".//div[@class='_4rR01T']")
        details["Brand Name"] = title.text.split()[0]
        details["Smartphone Name"] = title.text
        
        try:
            color = product.find_element(By.XPATH, ".//li[contains(text(), 'Color')]").text.split()[-1]
        except:
            color = "-"
        details["Colour"] = color
        
        details["RAM"] = get_detail(product, "RAM")
        details["Storage(ROM)"] = get_detail(product, "ROM")
        details["Primary Camera"] = get_camera_detail(product, "Primary Camera")
        details["Secondary Camera"] = get_camera_detail(product, "Front Camera")
        details["Display Size"] = get_detail(product, "Display Size")
        details["Battery Capacity"] = get_battery_detail(product, "Battery")
        
        price = product.find_element(By.XPATH, ".//div[@class='_30jeq3 _1_WHN1']").text
        details["Price"] = price
        
        product_url = product.find_element(By.XPATH, ".//a").get_attribute("href")
        details["Product URL"] = product_url
        
        data.append(details)
    
    # Close the web driver
    driver.quit()
    
    # Save the data in a data frame and a CSV file
    df = pd.DataFrame(data)
    df.to_csv(f"{smartphone}_search_results.csv", index=False)
    
    return df

def get_detail(product, label):
    try:
        return product.find_element(By.XPATH, f".//li[contains(text(), '{label}')]").text.split()[0]
    except:
        return "-"

def get_camera_detail(product, label):
    try:
        camera = product.find_element(By.XPATH, f".//li[contains(text(), '{label}')]").text.split()[0]
        if camera[-1] == "P":
            camera = camera[:-1]
        return camera + "MP"
    except:
        return "-"

def get_battery_detail(product, label):
    try:
        battery = product.find_element(By.XPATH, f".//li[contains(text(), '{label}')]").text.split()[0]
        if battery[-2:] == "Ah":
            battery = battery[:-2]
        return battery + " mAh"
    except:
        return "-"

# Example usage: search for Oneplus Nord on www.flipkart.com and scrape the details of the products listed on the first page of search results
df = search_flipkart("Oneplus Nord")
print(df)


   Brand Name                                    Smartphone Name Colour RAM  \
0     OnePlus      OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)      -   6   
1     OnePlus            OnePlus Nord 3 5G (Misty Green, 256 GB)      -  16   
2     OnePlus            OnePlus Nord 3 5G (Misty Green, 128 GB)      -   8   
3     OnePlus  OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 256...      -   8   
4     OnePlus     OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)      -   6   
5     OnePlus    OnePlus Nord CE 3 Lite 5G (Pastel Lime, 256 GB)      -   8   
6     OnePlus           OnePlus Nord 2T 5G (Gray Shadow, 128 GB)      -   8   
7     OnePlus           OnePlus Nord 2T 5G (Gray Shadow, 128 GB)      -   8   
8     OnePlus           OnePlus Nord 3 5G (Tempest Gray, 128 GB)      -   8   
9     OnePlus              OnePlus Nord 2T 5G (Jade Fog, 128 GB)      -   8   
10    OnePlus                    OnePlus Nord (Gray Onyx, 64 GB)      -   6   
11    OnePlus           OnePlus Nord 3 5G (Tempest G

In [ ]:
ques-5

In [17]:
# Set up the webdriver
driver = webdriver.Chrome()

# Navigate to the Google Maps website
driver.get("https://www.google.com/maps")

# Find the search box element and enter the city name
search_box = driver.find_element(By.XPATH, "//input[@id='searchboxinput']")
search_box.send_keys("New York City")
search_box.submit()

# Wait for the page to load
time.sleep(5)

# Get the current URL of the page
url = driver.current_url

# Close the webdriver
driver.close()

# Extract the latitude and longitude from the URL
coordinates = url.split('@')[1].split(',')[0:2]
latitude = coordinates[0]
longitude = coordinates[1]

# Print the latitude and longitude
print("Latitude:", latitude)
print("Longitude:", longitude)

Latitude: 40.9305088
Longitude: 29.212672


ques-6

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

# Set up the webdriver
driver = webdriver.Chrome()

# Navigate to the digit.in website
driver.get("https://www.digit.in/top-products/best-gaming-laptops-40.html")

# Find the elements containing the laptop details
laptop_elements = driver.find_elements(By.XPATH, "//div[@class='TopNumbeHeading active sticky-footer']")

# Create lists to store the laptop data
laptop_names = []
laptop_specs = []
laptop_prices = []

# Loop through the laptop elements and extract the details
for laptop_element in laptop_elements:
    # Extract the laptop name
    laptop_name = laptop_element.find_element(By.XPATH, ".//span[@class='data-featured-name']")
    laptop_names.append(laptop_name.text)

    # Extract the laptop specifications
    laptop_spec = laptop_element.find_element(By.XPATH, ".//div[@class='Specs-Wrap']")
    laptop_specs.append(laptop_spec.text)

    # Extract the laptop price
    laptop_price = laptop_element.find_element(By.XPATH, ".//span[@class='smprice']")
    laptop_prices.append(laptop_price.text)

# Close the webdriver
driver.close()

# Create a DataFrame from the scraped data
df = pd.DataFrame({
    'Laptop Name': laptop_names,
    'Laptop Specifications': laptop_specs,
    'Laptop Price': laptop_prices
})

# Display the DataFrame
print(df)


Empty DataFrame
Columns: [Laptop Name, Laptop Specifications, Laptop Price]
Index: []


ques-7

In [18]:
def scrape_forbes_billionaires():
    # Initialize the Chrome web driver
    driver = webdriver.Chrome()
    driver.get("https://www.forbes.com/billionaires/")
    
    time.sleep(5)  # Wait for the page to load
    
    # Find the list of billionaires
    billionaires_list = driver.find_elements(By.XPATH, "//div[@class='personName']")
    
    billionaire_details = []
    
    # Iterate through each billionaire and extract details
    for index, billionaire in enumerate(billionaires_list, start=1):
        details = {}
        
        # Click on the billionaire to reveal more details
        billionaire.click()
        time.sleep(2)  # Wait for details to load
        
        details["Rank"] = index
        details["Name"] = driver.find_element(By.XPATH, "//div[contains(@class,'profile-top')]//h1").text
        
        # Extract other details
        details_elements = driver.find_elements(By.XPATH, "//div[contains(@class,'profile-info')]//div[contains(@class,'data')]//div[contains(@class,'data-point')]")
        details["Net worth"] = details_elements[0].text
        details["Age"] = details_elements[1].text
        details["Citizenship"] = details_elements[2].text
        details["Source"] = details_elements[3].text
        details["Industry"] = details_elements[4].text
        
        billionaire_details.append(details)
        
        # Go back to the list of billionaires
        driver.back()
        time.sleep(2)
    
    driver.quit()
    return billionaire_details

if __name__ == "__main__":
    billionaires = scrape_forbes_billionaires()
    
    if billionaires:
        print("Billionaire details:")
        for billionaire in billionaires:
            print(billionaire)
    else:
        print("No billionaire details found.")


No billionaire details found.


In [ ]:
ques-9

In [19]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def scrape_hostel_data(location):
    # Initialize the Chrome web driver
    driver = webdriver.Chrome()
    driver.get(f"https://www.hostelworld.com/hostels/{location}")
    
    time.sleep(5)  # Wait for the page to load
    
    hostels = driver.find_elements(By.XPATH, "//div[@class='hostels-101__container']//div[@class='fabrating fabrating-small']")
    
    hostel_data = []
    
    for hostel in hostels:
        details = {}
        hostel.click()
        
        time.sleep(2)  # Wait for details to load
        
        details["Hostel Name"] = driver.find_element(By.XPATH, "//h1").text
        details["Distance from City Centre"] = driver.find_element(By.XPATH, "//span[contains(text(), 'km from city centre')]").text
        details["Ratings"] = driver.find_element(By.XPATH, "//div[contains(@class, 'score rating-summary')]/span/span").text
        details["Total Reviews"] = driver.find_element(By.XPATH, "//span[contains(text(), 'Total Reviews')]/following-sibling::span").text
        details["Overall Reviews"] = driver.find_element(By.XPATH, "//span[contains(text(), 'Overall Reviews')]/following-sibling::span").text
        details["Privates from Price"] = driver.find_element(By.XPATH, "//div[contains(text(), 'Privates From')]/following-sibling::span").text
        details["Dorms from Price"] = driver.find_element(By.XPATH, "//div[contains(text(), 'Dorms From')]/following-sibling::span").text
        details["Facilities"] = ", ".join([facility.text for facility in driver.find_elements(By.XPATH, "//div[@class='facilities']/div")])
        details["Property Description"] = driver.find_element(By.XPATH, "//div[contains(@class, 'property-description-content')]").text
        
        hostel_data.append(details)
        
        driver.back()
        time.sleep(2)
    
    driver.quit()
    return hostel_data

if __name__ == "__main__":
    location = "England/London"
    hostels_data = scrape_hostel_data(location)
    
    if hostels_data:
        df = pd.DataFrame(hostels_data)
        df.to_csv("hostels_data.csv", index=False)
        print("Hostel data saved to hostels_data.csv")
    else:
        print("No hostel data found.")


No hostel data found.
